# Longwave Radiation Transfer Example

This notebook demonstrates how to compute longwave radiation fluxes using the `pyrte_rrtmgp` library. It downloads the required data, computes the gas optics using an absorption approach, solves the radiation transfer equations, and validates the computed fluxes against reference datasets.

In [1]:
import os
import numpy as np
import xarray as xr
import dask

## Importing pyrte_rrtmgp Modules

The following modules from `pyrte_rrtmgp` are imported to handle gas optics and solve the radiation transfer equations.

In [2]:
from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver

ERROR_TOLERANCE = 1e-7

## Data Download and Directory Setup

Download the RRTMGP data and set up the directory paths for input and reference files.

In [3]:
rte_rrtmgp_dir = download_rrtmgp_data()
rfmip_dir = os.path.join(rte_rrtmgp_dir, "examples", "rfmip-clear-sky")
input_dir = os.path.join(rfmip_dir, "inputs")
ref_dir = os.path.join(rfmip_dir, "reference")
os.listdir(rte_rrtmgp_dir)

['rrtmgp-gas-lw-g256.nc',
 'LICENSE',
 'rrtmgp-clouds-lw-bnd.nc',
 'CITATION.cff',
 'rrtmgp-clouds-lw-g256.nc',
 'rrtmgp-aerosols-merra-lw.nc',
 'rrtmgp-clouds-sw-bnd.nc',
 'README.md',
 'rrtmgp-clouds-sw-g224.nc',
 'rrtmgp-gas-lw-g128.nc',
 '.gitignore',
 'rrtmgp-aerosols-merra-sw.nc',
 'examples',
 'rrtmgp-clouds-sw-g112.nc',
 'rrtmgp-clouds-lw-g128.nc',
 'rrtmgp-gas-sw-g224.nc',
 'rrtmgp-gas-sw-g112.nc']

## Loading Gas Optics and Atmosphere Dataset

Load the longwave gas optics file (`LW_G256`) and the corresponding atmosphere dataset for the simulation.

In [4]:
gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)

atmosphere_file = "multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
atmosphere_path = os.path.join(input_dir, atmosphere_file)
atmosphere = xr.open_dataset(atmosphere_path, chunks={"expt": 3})
atmosphere

<xarray.Dataset> Size: 2MB
Dimensions:                  (site: 100, expt: 18, layer: 60, level: 61)
Coordinates:
    lon                      (site) float32 400B dask.array<chunksize=(100,), meta=np.ndarray>
    lat                      (site) float32 400B dask.array<chunksize=(100,), meta=np.ndarray>
    time                     (site) datetime64[ns] 800B dask.array<chunksize=(100,), meta=np.ndarray>
    expt_label               (expt) <U49 4kB dask.array<chunksize=(3,), meta=np.ndarray>
Dimensions without coordinates: site, expt, layer, level
Data variables: (12/62)
    sst                      (site) float32 400B dask.array<chunksize=(100,), meta=np.ndarray>
    pres_layer               (site, layer) float32 24kB dask.array<chunksize=(100, 60), meta=np.ndarray>
    pres_level               (site, level) float32 24kB dask.array<chunksize=(100, 61), meta=np.ndarray>
    surface_emissivity       (site) float32 400B dask.array<chunksize=(100,), meta=np.ndarray>
    surface_albedo           (site) float32 400B dask.array<chunksize=(100,), meta=np.ndarray>
    solar_zenith_angle       (site) float32 400B dask.array<chunksize=(100,), meta=np.ndarray>
    ...                       ...
    hfc134a_GM               (expt) float32 72B dask.array<chunksize=(3,), meta=np.ndarray>
    c6f14_GM                 (expt) float32 72B dask.array<chunksize=(3,), meta=np.ndarray>
    hcfc141b_GM              (expt) float32 72B dask.array<chunksize=(3,), meta=np.ndarray>
    chcl3_GM                 (expt) float32 72B dask.array<chunksize=(3,), meta=np.ndarray>
    c2f6_GM                  (expt) float32 72B dask.array<chunksize=(3,), meta=np.ndarray>
    cfc11eq_GM               (expt) float32 72B dask.array<chunksize=(3,), meta=np.ndarray>
Attributes: (12/25)
    title:               Atmospheric conditions for off-line radiative transf...
    institution_id:      UColorado
    institution:         University of Colorado, Boulder, CO 80309, USA
    activity_id:         input4MIPs
    Conventions:         CF-1.6
    creation_date:       2019-03-20 16:07:21-0400
    ...                  ...
    nominal_resolution:  10 km
    target_mip:          RFMIP
    variable_id:         multiple
    grid_label:          none
    tracking_id:         hdl:21.14100/f379c294-f7bb-442d-bca7-64661d60780e
    license:             Atmospheric condition data for RFMIP produced by the...

## Computing Gas Optics and Solving Radiation Transfer Equations

Compute the gas optics using an absorption method and solve the longwave radiation transfer equations to obtain the fluxes.

In [5]:
gas_optics_lw.gas_optics.compute(atmosphere, problem_type="absorption")

solver = RTESolver()
fluxes = solver.solve(atmosphere, add_to_input=False)
fluxes

ValueError: dimension gas on 14th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. To fix, either rechunk into a single array chunk along this dimension, i.e., ``.chunk(dict(gas=-1))``, or pass ``allow_rechunk=True`` in ``dask_gufunc_kwargs`` but beware that this may significantly increase memory usage.

## Validating Computed Fluxes Against Reference Data

Load the reference datasets for upward and downward longwave fluxes and validate that the computed fluxes match within the specified error tolerance.

In [6]:
rlu_reference = f"{ref_dir}/rlu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rld_reference = f"{ref_dir}/rld_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rlu = xr.load_dataset(rlu_reference, decode_cf=False)
rld = xr.load_dataset(rld_reference, decode_cf=False)

assert np.isclose(fluxes["lw_flux_up"], rlu["rlu"], atol=ERROR_TOLERANCE).all(), "Longwave flux up mismatch"
assert np.isclose(fluxes["lw_flux_down"], rld["rld"], atol=ERROR_TOLERANCE).all(), "Longwave flux down mismatch"

print("Longwave radiation transfer calculations validated successfully!")

NameError: name 'fluxes' is not defined